In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ridership Open Lakehouse Demo (Part 3): Time-Series forecasting of ridership data

This notebook will demonstrate a strategy to implement an open lakehouse on GCP, using Apache Iceberg, as an open source standard for managing data, while still leveraging GCP native capabilities. This demo will use BigQuery Manged Iceberg Tables, Managed Apache Kafka and Apache Kafka Connect to ingest streaming data, Vertex AI for Generative AI queries on top of the data and Dataplex to govern tables.

This notebook will use the `bus_rides` data and ML models to generate a time-series forcasting of ridership in the future, in order to alert us when a bus about to become full.

We will evaluate the models accuracy and generate future data to be used in the next chapters for real-time predictions and alerting.

## Setup the environment

In [ ]:
import os
USER_AGENT = "cloud-solutions/data-to-ai-nb-v3"

PROJECT_ID = !gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
BQ_DATASET = "ridership_lakehouse"
BUCKET_NAME = f"{PROJECT_ID}-ridership-lakehouse"
LOCATION = "us-central1"
BQ_CONNECTION_NAME = "cloud-resources-connection"


print(PROJECT_ID)
print(BUCKET_NAME)

In [ ]:
from google.cloud import bigquery, storage
from google.api_core.client_info import ClientInfo

bigquery_client = bigquery.Client(
    project=PROJECT_ID,
    location=LOCATION,
    client_info=ClientInfo(user_agent=USER_AGENT)
)
storage_client = storage.Client(
    project=PROJECT_ID,
    client_info=ClientInfo(user_agent=USER_AGENT)
)

# Feature Engineering

Let's assume that bus ridership depends on a number of factors:
  * Station
  * Bus line
  * The Date and Time (time series)
  
  We will use the data generated in notebooks 1 & 2 to forecast ridership, based on these factors.

  Obviously, some of these variable are related (the station and Borough are realted). In a real-world scenario, this might be a bad practice, and might lead to overfitting towards a specific variable, but for the sake of the demo we will try to use these features.

### Create ridership table

In [ ]:
table_name = "bus_rides_features"

ridership_features_uri = f"gs://{BUCKET_NAME}/iceberg_data/{table_name}/"

bigquery_client.query(f"DROP TABLE IF EXISTS {BQ_DATASET}.{table_name};").result()
query = f"""
CREATE TABLE `{BQ_DATASET}.{table_name}`
WITH CONNECTION `{PROJECT_ID}.{LOCATION}.{BQ_CONNECTION_NAME}`
OPTIONS (
  file_format = 'PARQUET',
  table_format = 'ICEBERG',
  storage_uri = '{ridership_features_uri}'
)
AS
(
  SELECT
  r.timestamp_at_stop,
  r.bus_ride_id,
  r.bus_stop_id,
  r.bus_line_id,
  r.bus_size,
  r.total_capacity,
  s.borough,
  r.passengers_in_stop,
  r.passengers_boarding,
  r.passengers_alighting,
  r.remaining_capacity,
  r.remaining_at_stop,
  COALESCE(SAFE_DIVIDE(r.remaining_capacity, r.total_capacity), 0) AS remaining_capacity_percentage,
  COALESCE(SAFE_DIVIDE(r.remaining_at_stop, r.passengers_in_stop), 0) AS passengers_left_behind_percentage
FROM `{BQ_DATASET}.bus_rides` AS r
LEFT JOIN `{BQ_DATASET}.bus_stations` AS s ON s.bus_stop_id = r.bus_stop_id
);
"""
bigquery_client.query(query).result()

## Visualize Projected data

Let's now create a visualization for each of these features, the relationship with the `ridership` column, to see the effects of different features on the target variable.

### ridership per station over time
Let's take a look at the last 90 days of generated data. The first bus stop graph also shows the temperature and percipitation values.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import random

ridership_history = bigquery_client.query(f"""
DECLARE max_ts TIMESTAMP DEFAULT (SELECT MAX(timestamp_at_stop) FROM ridership_lakehouse.{table_name});
SELECT *
  FROM `ridership_lakehouse.{table_name}`
  WHERE timestamp_at_stop > TIMESTAMP_SUB(max_ts, INTERVAL 90 DAY)
""").result().to_dataframe()

random.seed(42)

# we'll sample 20 random stations, as displaying all of them is not practical
bus_line_ids = random.sample(list(ridership_history.bus_line_id.unique()), k=20)

figure = plt.figure(figsize=(20, 6))
plt.xlabel('Timestamp at stop')
# Group data by station ID
for bus_line_id in bus_line_ids:
  station_data = ridership_history[ridership_history['bus_line_id'] == bus_line_id].sort_values(by="timestamp_at_stop")
  # Plot ridership over time for the current station
  plt.plot(station_data['timestamp_at_stop'], station_data['remaining_at_stop'], label=f'Bus Line {bus_line_id}')

# Customize the plot
plt.xlabel('Timestamp at stop')
plt.ylabel('Remaining Passengers')
plt.title('Remaining Passengers Over Time by Bus Line')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
del ridership_history  # save on ram space

### Avg ridership per Borough

In [ ]:
average_ridership_per_borough = bigquery_client.query("""
-- select a random sample of data, group by `borough` and calculate the average of ridership
SELECT
  borough,
  AVG(ridership) AS average_ridership
FROM
  `ridership_lakehouse`.`ridership_features`
TABLESAMPLE
  SYSTEM(10 PERCENT)
GROUP BY
  borough;
""").result().to_dataframe()

# Sort by average ridership to ensure correct order in the plot
average_ridership_per_borough = average_ridership_per_borough.sort_values(by='average_ridership').reset_index()

average_ridership_per_borough.plot.bar(y='average_ridership', x='borough', rot=0)

### Average ridership per month

In [ ]:
# Generate New Visualization: Average Ridership per Month -- we'll select a new preojection

# Calculate average ridership per month
# Group by 'transit_month' and calculate the mean of 'ridership'
average_ridership_per_month = bigquery_client.query("""
-- select a random sample of data, group by `transit_month` and calculate the average of ridership
SELECT
  transit_month,
  AVG(ridership) AS average_ridership
FROM
  `ridership_lakehouse`.`ridership_features`
TABLESAMPLE
  SYSTEM(10 PERCENT)
GROUP BY
  transit_month;
""").result().to_dataframe()

# Sort by month to ensure correct chronological order in the plot
average_ridership_per_month = average_ridership_per_month.sort_values(by='transit_month').reset_index()

plt.figure(figsize=(10, 6)) # Adjust figure size

plt.plot(average_ridership_per_month['transit_month'], average_ridership_per_month['average_ridership'], marker='o', linestyle='-', color='skyblue')

# Add plot labels and title
plt.xlabel("Transit Month")
plt.ylabel("Average Ridership")
plt.title("Average Transit Minutly Ridership per Month")
plt.xticks(average_ridership_per_month['transit_month']) # Ensure all months are shown on x-axis
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
del average_ridership_per_month  # save on RAM

### Average Ridership per Day-of-Week

In [ ]:
# Generate New Visualization: Average Ridership per Day-of-Week

# Calculate average ridership per day of week
# Group by 'transit_day_of_week' and calculate the mean of 'ridership'
average_ridership_per_day_of_week = bigquery_client.query("""
-- select a random sample of data, group by `transit_day_of_week` and calculate the average of ridership
SELECT
  transit_day_of_week,
  AVG(ridership) AS average_ridership
FROM
  `ridership_lakehouse`.`ridership_features`
TABLESAMPLE
  SYSTEM(10 PERCENT)
GROUP BY
  transit_day_of_week;
""").result().to_dataframe()

# Sort by month to ensure correct chronological order in the plot
average_ridership_per_day_of_week = average_ridership_per_day_of_week.sort_values(by='transit_day_of_week').reset_index()

days_of_week = {
    1: "Sunday",
    2: "Monday",
    3: "Tuesday",
    4: "Wednesday",
    5: "Thursday",
    6: "Friday",
    7: "Saturday"
}
average_ridership_per_day_of_week['transit_day_of_week'] = average_ridership_per_day_of_week['transit_day_of_week'].map(days_of_week)
plt.figure(figsize=(10, 6)) # Adjust figure size

plt.plot(average_ridership_per_day_of_week['transit_day_of_week'], average_ridership_per_day_of_week['average_ridership'], marker='o', linestyle='-', color='skyblue')

# Add plot labels and title
plt.xlabel("Transit Day of Week")
plt.ylabel("Average Ridership")
plt.title("Average Transit Minutly Ridership per Day of Week")
plt.xticks(average_ridership_per_day_of_week['transit_day_of_week']) # Ensure all months are shown on x-axis
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
del average_ridership_per_day_of_week  # save on RAM

# Forecast bus ridership

For time-series forecasting, there are 3 main options to choose from.

### ARIMA_PLUS
This model is an enhanced version of the traditional ARIMA, offering improved performance and often including automatic parameter selection for better ease of use in forecasting univariate time series. It's a **Univariate** model, meaning, it is built to predict a target value based on only the timestamp variable. You cannot incorportate extra variables that might influence the prediction. For that reason, we will **NOT** use it here in our demo.

### ARIMA_PLUS_XREG
Extending `ARIMA_PLUS`, this model incorporates the ability to utilize external regressors (exogenous variables) to enhance the forecasting accuracy by accounting for the influence of other relevant time series. Like the `ARIMA_P{LUS` It still relies on linearity assumptions between regressors and target. It is a Multivariate model, in terms of inputs, but still forecasts a single output.)

### TimesFM
`TimesFM` is a deep learning-based forecasting model that leverages transformer architectures to capture complex temporal dependencies and long-range patterns in time series data, often excelling in multi-variate forecasting tasks. It's considered excellent at capturing complex non-linear patterns and long-range dependencies; naturally handles multivariate inputs and outputs; can perform well with large datasets.It can be, however, prone to overfitting with small datasets.

## What's next?

In the rest of this notebook, we will create predication using both `ARIMA_PLUS_XREG` and `TimesFM` and evaluate each, so we can comopare the performance of each of the models.


## Multivariate forecasting using the ARIMA_PLUS_XREG model

### Train the model


The same CREATE MODEL statement is used to train this model Many options, e.g,  `time_series_data_col`, `time_series_timestamp_col`,  `time_series_id_col` have the same meaning as for the ARIMA_PLUS model.

The main difference - the ARIMA_PLUS_XREG model uses all columns besides those identified by the options above as the feature columns and uses linear regression to calculate covariate weights.

For details on the additional options, explanation of the training process, and best practices when training and using the model please refer to BigQuery documentation on [the CREATE MODEL statement for ARIMA_PLUS_XREG models](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-multivariate-time-series).

In [ ]:
%%bigquery
DECLARE min_ts TIMESTAMP DEFAULT TIMESTAMP("2023-09-01T00:00:00");
DECLARE max_ts TIMESTAMP DEFAULT TIMESTAMP("2024-08-31T23:59:59");

CREATE OR REPLACE MODEL `ridership_lakehouse.ridership_arima_plus_xreg`
OPTIONS(
  MODEL_TYPE = 'ARIMA_PLUS_XREG',
  TIME_SERIES_ID_COL = ['station_id'],
  TIME_SERIES_DATA_COL = 'ridership',
  TIME_SERIES_TIMESTAMP_COL = 'transit_timestamp',
  AUTO_ARIMA=TRUE,
  -- DATA_FREQUENCY='MONTHLY',
  HOLIDAY_REGION = "US"  -- the original dataset is from NY
)
AS SELECT
  station_id,
  borough,
  transit_month,
  transit_day_of_week,
  transit_timestamp,
  ridership
FROM `ridership_lakehouse.ridership_features`
TABLESAMPLE
  SYSTEM(70 PERCENT)
WHERE transit_timestamp >= min_ts AND transit_timestamp < max_ts; -- we're only training

### Forecast ridership

Let's move the latest forecast data to the location of the model:

In [ ]:
%%bigquery --project {PROJECT_ID}

DROP TABLE IF EXISTS multimodal.latest_local_forecast;

In [ ]:
! bq cp -f -n '{PROJECT_ID}:weathernext_graph_derived.latest_local_forecast' '{PROJECT_ID}:multimodal.latest_local_forecast'

#### Run the time-series forecast

If we were to use the ARIMA_PLUS model we could have just run the ML.FORECAST function to get time-series predictions. But the ARIMA_PLUS_XREG assumes that additional features will affect the forecast and you must provide the expected features to the ML.FORECAST function.

We already have most of the parts to prepare the features. Let's first test the feature preparation SQL before running the model.



In [ ]:
%%bigquery expected_features --project {PROJECT_ID}

DECLARE time_zone DEFAULT "America/New_York";

-- Forecast from now...
DECLARE start_ts DEFAULT CURRENT_TIMESTAMP();
-- to 5 days forward
DECLARE end_ts DEFAULT TIMESTAMP_ADD(start_ts, INTERVAL 5 DAY);

WITH
event_timestamps AS (
  SELECT TIMESTAMP(DATETIME(event_ts_in_utc, time_zone)) event_ts FROM
    UNNEST(GENERATE_TIMESTAMP_ARRAY(start_ts, end_ts, INTERVAL 5 MINUTE)) as event_ts_in_utc
),
bus_stops_and_event_timestamps AS (
  -- Cartesian join of the bus stops and time points
  -- We only need the bus stop ids and locations, not all the meta data
  SELECT bus_stops.bus_stop_id, bus_stops.location, event_ts
    FROM multimodal.bus_stops, event_timestamps
),
events_and_weather AS (
  SELECT
    bus_stop_id,
    event_ts,
    weather.forecast,
    multimodal.temperature_approx(weather.forecast, event_ts) as temperature,
    -- we are getting the latest forecast data, not historical
    FROM bus_stops_and_event_timestamps events, multimodal.latest_local_forecast weather
      WHERE ST_COVERS(weather.geography_polygon, events.location) AND
        event_ts BETWEEN TIMESTAMP_SUB(weather.forecast.time, INTERVAL 6 HOUR) AND weather.forecast.time
)
SELECT
    bus_stop_id,
    event_ts,
    -- the two features used by the model
    temperature,
    forecast.total_precipitation_6hr as total_precipitation_6hr,
    FROM events_and_weather
    -- we are going to drop these clauses later, this is just to help with visualization
    ORDER by bus_stop_id, event_ts
    LIMIT 50;

Let's see what these features look like:

In [ ]:
display(expected_features)

OK, the features look correct. One nuance - you might see that the temperature values are unchanged for some earlier event timestamps. This is because our latest forecast table doesn't have temperature values for earlier forecasts and the temperature approximation function just takes the current value if there is no earlier one. We can find this data if needed by looking at the previous forecast, but that would result in more complex SQL statement.

Let's run the forecast. We will use most of the feature preparation SQL in the forecasting function. Alternatively, we could have prepared the features and saved them in a table and used the whole table as the feature input to the forecast function.

In [ ]:
%%bigquery ridership_forecast --location {REGION}

DECLARE time_zone DEFAULT "America/New_York";

-- Forecast from now...
DECLARE start_ts DEFAULT CURRENT_TIMESTAMP();
-- to 5 days forward
DECLARE end_ts DEFAULT TIMESTAMP_ADD(start_ts, INTERVAL 5 DAY);

SELECT
  *
FROM
  ML.FORECAST (
    model `multimodal.ridership_arima_plus_xreg`,
    STRUCT (1000 AS horizon, 0.8 AS confidence_level),
    (
WITH
event_timestamps AS (
  SELECT TIMESTAMP(DATETIME(event_ts_in_utc, time_zone)) event_ts FROM
    UNNEST(GENERATE_TIMESTAMP_ARRAY(start_ts, end_ts, INTERVAL 5 MINUTE)) as event_ts_in_utc
),
bus_stops_and_event_timestamps AS (
  -- Cartesian join of the bus stops and time points
  -- We only need the bus stop ids and locations, not all the meta data
  SELECT bus_stops.bus_stop_id, bus_stops.location, event_ts FROM multimodal.bus_stops, event_timestamps
),
events_and_weather AS (
  SELECT
    bus_stop_id,
    event_ts,
    weather.forecast,
    multimodal.temperature_approx(weather.forecast, event_ts) as temperature,
    -- we are getting the latest forecast data, not historical
    FROM bus_stops_and_event_timestamps events, multimodal.latest_local_forecast weather
      WHERE ST_COVERS(weather.geography_polygon, events.location) AND
        event_ts BETWEEN TIMESTAMP_SUB( weather.forecast.time, INTERVAL 6 HOUR) AND weather.forecast.time
)
SELECT
    bus_stop_id,
    event_ts,
    -- the two features used by the model
    temperature,
    forecast.total_precipitation_6hr as total_precipitation_6hr,
    FROM events_and_weather    )
  );

#### Visualize the forecast

Here's what the ML.FORECAST function returns:

In [ ]:
display(ridership_forecast)

Let's visualize the forecast:

In [ ]:

bus_stop_list = list(ridership_history.bus_stop_id.unique())
bus_stop_list.sort()

for bus_stop_id in bus_stop_list:

    historical_data = ridership_history[ridership_history.bus_stop_id==bus_stop_id]
    forecast_data = ridership_forecast[ridership_forecast.bus_stop_id==bus_stop_id]
    plot_historical_and_forecast(historical_data = historical_data,
                                 timestamp_col_name = "event_ts",
                                 data_col_name = "num_riders",
                                 forecast_output = forecast_data,
                                 title = bus_stop_id)

# Selecting the right model

In the previous sections we have seen the mechanics of creating and using two time-series forecasting models. The univariate model is simpler to create and is simpler to use for forecasting. The multivariate model can give more accurate forecasting. In a real production implementation it would be important to capture the prediction results and compare with the actual outcomes. This would help to decide if it's worth building the multivariate model, fine tune other model parameters - training data period, additional features, effect of holidays and seasonality, etc.

Part of the model selection process is model evaluation and forecast evaluation.  

## Model evaluation

In [ ]:
%%bigquery arima_plus_model_evaluation --project {PROJECT_ID}

SELECT
  *
FROM
  ML.ARIMA_EVALUATE(MODEL multimodal.ridership_arima_plus, STRUCT(FALSE AS show_all_candidate_models))

In [ ]:
display_columns_as_rows(arima_plus_model_evaluation)

You can see that there are multiple time series models under the over, one for each bus stop. For details on how to interpret the output of ML.ARIMA_EVALUATE function refer to the [BigQuery documentation](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-arima-evaluate).

## Forecast explanation

To evaluate the forecast, use the same parameters as use for ML.FORECAST function to call ML.EXPLAIN_FORECAST:

In [ ]:
%%bigquery arima_plus_forecast_explanation --project {PROJECT_ID}

SELECT *
 FROM ML.EXPLAIN_FORECAST(
  MODEL multimodal.ridership_arima_plus,
  STRUCT(300 AS horizon, 0.8 AS confidence_level))

The output of the function contains two types of records - the ones that were used for trainging and the actual forecast.

Here are a couple of records of historical records:

In [ ]:
data_to_show = arima_plus_forecast_explanation[
    (arima_plus_forecast_explanation['bus_stop_id'] == 'bus-stop-1') &
    (arima_plus_forecast_explanation['time_series_type'] == 'history')]


display_columns_as_rows(data_to_show.head(2))

And here are some fo the forecast records:

In [ ]:
data_to_show = arima_plus_forecast_explanation[
    (arima_plus_forecast_explanation['bus_stop_id'] == 'bus-stop-1') &
    (arima_plus_forecast_explanation['time_series_type'] == 'forecast')]


display_columns_as_rows(data_to_show.head(2))

For details on how to interpret the output of the function refer to the [BigQuery documentation](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-explain-forecast).

# Conclusion

We showed how you can use two different time-series forecasting models available in BigQuery.

We also showed how the WeatherNext Graph dataset can be used to get historical and future weather forecasts.

There are multiple use cases for these time-series forecasts. For example, an AI agent can use the results of ridership forecast as an input to decision on how to perform a task like this: "In the next week schedule the repair of the bus stop #2 during the least impactful to passengers time".